In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np

from sklearn.metrics import ndcg_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
import lightgbm as lgb

In [2]:
# предварительный ранкер не хотелось делать сложным и переобученным
ranker_model = lgb.LGBMRanker(n_estimators = 20,
                         random_state = 33,
                         n_jobs = 8
                         )

study_cols = ['cnt_users_by_item',
       'mean_time_by_item', 'mean_good_by_item', 'mean_abs_react_by_item',
       'pretarget_time_sum_5m', 'pretarget_time_sum_1m',
       'pretarget_good_sum_5m', 'pretarget_good_sum_1m', 'pretarget_prc',
       'cnt_items', 'time_sum', 'good_mean', 'good_sum', 'reaction_mean',
       'reaction_abs_mean', 'reaction_abs_sum', 'als_score', 'emb_als_score',
       'emb_als_score_tune', 'cosine', 'source_good_mean',
       'source_good_sum']

In [3]:
import gc
result_lst = []
for i in range(2):
    tmp_result_df = pd.read_parquet(f'/srv/data/vk/old/result_df_{i}.parquet.gzip')
    tmp_result_df['user_id'] = tmp_result_df['user_id']+3000000
    result_lst.append(tmp_result_df)
    tmp_result_df = pd.read_parquet(f'/srv/data/vk/train/result_df_{i}.parquet.gzip')
    result_lst.append(tmp_result_df)
    gc.collect()

In [4]:
result_df = pd.concat(result_lst).reset_index(drop=True)
result_df.loc[result_df['timespent']>10,'timespent'] = 10
result_df = result_df.sort_values('user_id').reset_index(drop=True)
group_pretrain = result_df.groupby('user_id').size().reset_index(name='cnt').cnt.values

In [6]:
%%time

ranker_model.fit(result_df[study_cols], 
          result_df['timespent'], 
          group=group_pretrain
         )

CPU times: user 11min 49s, sys: 17.7 s, total: 12min 7s
Wall time: 1min 56s


LGBMRanker(n_estimators=20, n_jobs=8, random_state=33)

In [7]:
import pickle
with open('pre_ranker_final.pickle', 'wb') as f:
    pickle.dump(ranker_model, f)